<a href="https://colab.research.google.com/github/Koowater/goorm-Magicians/blob/main/KLUE_self_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###KLUE 정답과 편집거리 계산 

캐글 제출시 편집거리가 어느정도 나올지 예측

편집거리 함수

In [ ]:
def edit(s1, s2, debug=False):
    if len(s1) < len(s2):
        return edit(s2, s1, debug)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))

        if debug:
            print(current_row[1:])

        previous_row = current_row

    return previous_row[-1]

KLUE dataset 불러옴

In [ ]:
kl_dataset = KoMRC.load(join(data_dir, 'klue-mrc-v1.1_dev.json'))

In [ ]:
from statistics import mean

In [ ]:
with torch.no_grad():
    edit_ds=[]
    for idx, data in enumerate(kl_dataset): 
        if verbose: print(f'질문: {data["question"]}')
        tokenized_example = tokenizer(data['question'], 
                                      data['context'], 
                                      truncation="only_second",
                                      max_length=512,
                                      stride=128,   #-#
                                      return_overflowing_tokens=True,
                                      return_offsets_mapping=True,
                                      padding='max_length'
                                      )
        input_ids, token_type_ids = [
            torch.tensor(tokenized_example[key], dtype=torch.long, device="cuda")
            for key in ("input_ids", "token_type_ids")
        ]
        offset_mapping = tokenized_example['offset_mapping']

        start_logits, end_logits = model(input_ids=input_ids, token_type_ids=token_type_ids, return_dict=False)
        start_logits = start_logits.cpu()
        end_logits = end_logits.cpu()

        
        if kl_dataset[idx]["answers"]==[]: #kl dataset은 정답이 없는 문제가 있음 패스
          continue
        else:
          answer=kl_dataset[idx]["answers"][0]["text"] 
          predict = postprocessor.eval(input_ids.cpu(), start_logits, end_logits, data['context'], offset_mapping, max_len=24, verbose=verbose) #-# max_len만 바꿔주세요 40, 20, etc
          edit_ds.append(edit(predict, answer))
        

print(f"kLUE_edit_distance: {mean(edit_ds):.3f}") 

###max_len에 따른 편집거리 변화 그래프

가장 짧은 편집거리를 내는 max_len을 계산 

시간이 오래걸림 원하는 범위만 확인

In [ ]:
verbose=False
output_path = join('/content/drive/MyDrive/Colab Notebooks/kaggle', 'split train.csv')
x=range(1, 50) #-# 원하는 범위만 좁혀서 볼 것
leven=[]
rows=[]
for i in x:
  with torch.no_grad():
    edit_ds=[]
    for idx, data in enumerate(kl_dataset): #-#
        if verbose: print(f'질문: {data["question"]}')
        tokenized_example = tokenizer(data['question'], 
                                      data['context'], 
                                      truncation="only_second",
                                      max_length=512,
                                      stride=64, 
                                      return_overflowing_tokens=True,
                                      return_offsets_mapping=True,
                                      padding='max_length'
                                      )
        input_ids, token_type_ids = [
            torch.tensor(tokenized_example[key], dtype=torch.long, device="cuda")
            for key in ("input_ids", "token_type_ids")
        ]
        offset_mapping = tokenized_example['offset_mapping']

        start_logits, end_logits = model(input_ids=input_ids, token_type_ids=token_type_ids, return_dict=False)
        start_logits = start_logits.cpu()
        end_logits = end_logits.cpu()
        
        if kl_dataset[idx]["answers"]==[]:
          continue
        else:
          answer=kl_dataset[idx]["answers"][0]["text"]
          predict = postprocessor.eval(input_ids.cpu(), start_logits, end_logits, data['context'], offset_mapping, max_len=i, verbose=verbose)
          edit_ds.append(edit(predict, answer))


    leven.append(mean(edit_ds))
    min_leven_dist=min(leven)
    id=leven.index(min_leven_dist)
    rows.append([i, mean(edit_ds)])
    print(leven)
    print("가장 짧은 편집거리:", min_leven_dist)
    print("가장 짧은 편집거리를 내는 max_len:", x[id])

df = pd.DataFrame(rows, columns=['max_len', 'edit distance'])
df.to_csv(output_path, index=False, sep=',', encoding='utf-8')

그래프

In [ ]:
# from matplotlib import pyplot as plt
# y = leven
# plt.plot(x, y)
# plt.xlabel("max_len")
# plt.ylabel("levenshtein distance")
# plt.title("kl_dataset")
# plt.show()